# INDRA portal outputs
Preparing 2D (lat,lon) netcdf files for AI, SPI and rainfall 15th percentile for input to ACS INDRA web portal. Multi-model median values for each index are provided for each GWL and change.

In [1]:
# import needed packages
import xarray as xr
import numpy as np
import lib_david
import pandas as pd

# Suppress all warnings
import warnings
warnings.filterwarnings('ignore')

#< Keep attributes from input datasets
xr.set_options(keep_attrs = True)

In [2]:
root_dir = "/g/data/ia39/ncra/drought_aridity/"
gwl = [1.2,1.5,2.0,3.0]

#< AGCD masking
mask_ds_nhp1 = xr.open_dataset("/g/data/mn51/users/dh4185/agcd_v1-0-2_precip_weight_r005_daily_1975_2005fraction_ge1.nc").sel(lat=slice(-44,-10),lon=slice(112,154))
q_mask_nhp1 = np.ma.masked_greater(mask_ds_nhp1.fraction,0.8)

mask_ds_acs = xr.open_dataset("/g/data/mn51/users/dh4185/agcd_v1-0-2_precip_weight_r005_daily_1960_2022fraction_ge1.nc")
q_mask_acs = np.ma.masked_greater(mask_ds_acs.fraction,0.8)

In [9]:
# process AI absolute values
for gwl_ in gwl:
    ds = xr.open_dataset(f"{root_dir}/ai/AI-atmospheric-based_NHP1-AUS-5_rcp85_bias-adjusted_2D_GWL{int(gwl_*10)}.nc")
    da = ds.median("model").squeeze().where(q_mask_nhp1.mask)
    del da.attrs["bias correction"]
    da.attrs["cell_methods"] = "multi-model median"
    da.where(q_mask_nhp1.mask).to_netcdf(f"{root_dir}/INDRA_portal/AI-atmospheric-based_NHP1-AUS-5_rcp85_bias-adjusted_2D_GWL{int(gwl_*10)}_MM50p_INDRA.nc")

In [10]:
# process SPI
for gwl_ in gwl:
    ds = xr.open_dataset(f"{root_dir}/spi/downscaled_BC_5km/SPI3_pct_time_below_-1_GWL_{gwl_}_ACS-QME-AGCD-1960-2022_percentiles_10-50-90.nc")
    da = ds.sel(quantile=0.5).where(q_mask_acs.mask)
    da.attrs["cell_methods"] = "multi-model median"
    da.to_netcdf(f"{root_dir}/INDRA_portal/SPI3_pct_time_below_-1_GWL_{gwl_}_ACS-QME-AGCD-1960-2022_MM50p_INDRA.nc")

In [11]:
# process 15th rainfall percentile
for gwl_ in gwl:
    ds = xr.open_dataset(f"{root_dir}/rainfall_percentiles/downscaled_BC_5km/p15_3month_GWL_{gwl_}_ACS-QME-AGCD-1960-2022_percentiles_10-50-90.nc")
    da = ds.mean("month").sel(quantile=0.5).where(q_mask_acs.mask)
    da.attrs["cell_methods"] = "multi-model median"
    da.to_netcdf(f"{root_dir}/INDRA_portal/p15_3month_GWL_{gwl_}_ACS-QME-AGCD-1960-2022_MM50p_INDRA.nc")
